In [1]:
#!pip install einops pytorch torchvision numpy sklearn segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 887.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [1]:
!pip install einops segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 843.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.8 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=62688ae687f8d88789327978c2767015611e829593f34d1923ca2564d05f06b4
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=2790a14234967f495ea839afc7d2a537ff91b8138e4f00820ede7010abcdd64d
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-p

In [2]:
!git clone https://github.com/axelden/Few-shot-DIA-WACV2023

Cloning into 'Few-shot-DIA-WACV2023'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 205 (delta 8), reused 5 (delta 0), pack-reused 181
Receiving objects: 100% (205/205), 259.08 MiB | 38.96 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
%cd Few-shot-DIA-WACV2023

/content/Few-shot-DIA-WACV2023


In [4]:
!python ./train.py


starting model training...
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in orde

In [9]:
import torch
from einops import rearrange
import time
import numpy as np
from BibDataset2 import Bib_testset
from torchvision import transforms
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from sklearn.metrics import confusion_matrix


import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import cv2
import torch
from skimage.measure import label, regionprops, regionprops_table
import torch.nn as nn

def saveIMGRes(imgs, masks, save_path):
    # creating the colormaps
    colour_table = np.zeros((256, 1, 3), dtype=np.uint8)
    colour_table[0] = [0, 0, 0]
    colour_table[1] = [0, 255, 255]
    colour_table[2] = [255, 255, 0]
    colour_table[3] = [255, 0, 255]

    # iterating throug the images and masks
    for img, mask in zip(imgs, masks):
        date_string = time.strftime("%d%m%Y-%H-%M-%S")
        # applying colormaps to the masks
        colour_mask = cv2.applyColorMap(mask.squeeze().astype(np.uint8), colour_table)
        # formatting image
        np_img = (img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        # adding generated masks to the images
        img = cv2.addWeighted(np_img, 0.1, colour_mask, 1, 0)
        # saving results
        cv2.imwrite(save_path + f"IMG{date_string}.png", img)

def get_scores( y_true,y_pred, average="macro"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=average)
    recall = recall_score(y_true, y_pred, average=average)
    f1_sc = f1_score(y_true, y_pred, average=average)
    iou_sc = jaccard_score(y_true, y_pred, average=average)
    return accuracy, precision, recall, f1_sc, iou_sc






###################################################################
################## HYPERPARAM AND COSTANTS ########################
#############6######################################################
import warnings
warnings.filterwarnings('ignore')
#IMG_W =1120
# PATCH_SIZE =224
#IMG_H = 1344
IMG_W =1344
IMG_H = 2016
PATCH_SIZE=336
PATCH_SPLIT=True
GEN_CROPS=False
WINDOW_STEP = PATCH_SIZE
H_PATCHES = int(IMG_H//PATCH_SIZE)
W_PATCHES = int(IMG_W//PATCH_SIZE)
DS = 'CB55'
BACKBONE = "resnet50"
date_string = time.strftime("%d%m%Y-%H-%M-%S")
REFINED_MASKS = True
SAVE_IMGS = True

###################################################################
####################### DATA LOADING ##############################
###################################################################
tf = transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Resize((IMG_H,IMG_W)),
])

device = "cuda"

###################################################################
###################################################################
model = smp.DeepLabV3Plus(encoder_name=BACKBONE, encoder_weights=None, classes=4).to(device)
model.eval()
LOAD_MODEL = True
DS="CB55"

SAVE_PATH = f"results/{DS}/"
data_path = f'fewshot_data/{DS}/img-{DS}'
data_path_GT = f'fewshot_data/{DS}/pixel-level-gt-{DS}'
#data_path = f'test'
#data_path_GT = f'test'
MODEL_LOAD_PATH = f'./trained_models/{DS}/_imgW={IMG_W}_imgH={IMG_H}_dlv3+336_bb=resnet50_fewshot_dilation.pt'
test_data = Bib_testset(data_path, data_path_GT, tf, gen_crops=GEN_CROPS, patch_split=PATCH_SPLIT, patch_size=(PATCH_SIZE, PATCH_SIZE))
test_loader = DataLoader(test_data, batch_size=1)
model.to(device)
if LOAD_MODEL:
  model.load_state_dict(torch.load(MODEL_LOAD_PATH))
  print("\nstarting...")
  all_gts =np.array([])
  all_preds =np.array([])
  for i, batch in enumerate(test_loader):
        t0 = time.time()
        data = []
        # loading and formatting the data
        patches = batch["images"].to(device)
        gts = batch["gt"]
        refinement_mask = batch["bin_mask"]
        if PATCH_SPLIT:
            patches = rearrange(patches, "b n c h w -> (b n) c h w")
            gts = rearrange(gts, "b n h w -> (b n) h w")
            refinement_mask = rearrange(refinement_mask, "b n h w -> (b n) h w")

        # running the model and obtaining the predicted classes
        out = model(patches.float())
        patch_predictions = out.detach().cpu().argmax(1)

        # mask refinement process
        if REFINED_MASKS:
            patch_predictions = (patch_predictions*refinement_mask)

        all_gts = np.concatenate([all_gts, gts.numpy().flatten()])
        all_preds = np.concatenate([all_preds, patch_predictions.numpy().astype(int).flatten()])

        t1 = time.time() - t0
        # print(f'-- Epoch: {i:3}, Time: {t1:6.2f} --')
        if SAVE_IMGS:
            print("printing imgs")
            bin_mask_rgb = np.repeat(refinement_mask.unsqueeze(1), 3, 1)
            masked_patches = patches.cpu() * bin_mask_rgb
            masked_imgs = rearrange(masked_patches, "(b h w) c p1 p2 -> b c (h p1) (w p2)", h=H_PATCHES, w=W_PATCHES)
            imgs_predictions = rearrange(patch_predictions, "(b h w c) p1 p2 -> b c (h p1) (w p2)", c=1, h=H_PATCHES, w=W_PATCHES)
            saveIMGRes(masked_imgs, imgs_predictions.numpy(), SAVE_PATH)

  print(f"############## {DS} Scores ##############")
  w_acc, w_prec, w_rec, w_f1_sc, w_iou_sc = get_scores(all_gts, all_preds, average="weighted")
  w_acc_macro, w_prec_macro, w_rec_macro, w_f1_sc_macro, w_iou_sc_macro = get_scores(all_gts, all_preds,
                                                                                       average="macro")
  print("weighted average scores:")
  # print(np.mean(weight_acc))
  print(np.mean(w_prec))
  print(np.mean(w_rec))
  print(np.mean(w_iou_sc))
  print(np.mean(w_f1_sc))
  print("macro average scores:")
    # print(np.mean(weight_acc))
  print(np.mean(w_prec_macro))
  print(np.mean(w_rec_macro))
  print(np.mean(w_iou_sc_macro))
  print(np.mean(w_f1_sc_macro))

    # cm = np.round(confusion_matrix(all_gts, all_preds, normalize="true"), 2)
    # print(cm)


starting...
printing imgs
printing imgs
printing imgs
printing imgs
printing imgs
printing imgs
printing imgs
printing imgs
printing imgs
printing imgs
############## CB55 Scores ##############
weighted average scores:
0.9865140261220362
0.9883869150958995
0.9773641071735605
0.9874266472530577
macro average scores:
0.7254980732500592
0.7160756745357536
0.6944098614823635
0.7207076381733877
